<a href="https://colab.research.google.com/github/maleehahassan/HIDA_Into_to_DL/blob/main/02_monitor_training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Overview: Predicting Sunshine Hours Tomorrow with Neural Networks

This notebook demonstrates a regression task using a weather prediction dataset. The goal is to predict the number of sunshine hours in Basel for the next day using historical weather data and neural networks. The workflow includes:

- **Data Loading & Exploration**: The weather dataset is loaded from a remote CSV file. Summary statistics, shape, and column names are explored to understand the data structure.
- **Feature Selection**: Features are selected by dropping date-related columns. The target variable is set as the next day's sunshine hours in Basel.
- **Data Splitting**: The data is split into training, validation, and test sets to enable model evaluation and prevent overfitting.
- **Model Building**: A feedforward neural network is constructed using TensorFlow/Keras, with customizable hidden layers for regression.
- **Training & Monitoring**: The model is trained for multiple epochs, and training progress is monitored using RMSE (Root Mean Squared Error).
- **Evaluation & Visualization**: Model predictions are compared to true values using scatter plots and RMSE metrics. Overfitting is discussed and addressed by adjusting model complexity and using early stopping.
- **Baseline Comparison**: A simple baseline predictor (yesterday's sunshine hours) is used for comparison to assess model performance.
- **Further Techniques**: Suggestions for additional improvements, such as batch normalization and dropout layers, are provided.

This notebook guides learners through the process of preparing data, building and training a neural network for regression, evaluating results, and improving model generalization on a real-world weather dataset.

# Predicting the Sunshine Hours Tomorrow

A regression task on the weather prediction dataset



In [ ]:
# Import pandas for data manipulation
import pandas as pd

# Load the weather prediction dataset from a remote CSV file
df = pd.read_csv("https://zenodo.org/record/5071376/files/weather_prediction_dataset.csv")

# Display summary statistics of the dataframe
df.describe()

In [ ]:
# Show the shape (rows, columns) of the dataframe
df.shape

In [ ]:
# List all column names in the dataframe
df.columns

In [ ]:
# Print the set of unique suffixes in column names (excluding 'MONTH' and 'DATE')
print(set({ "_".join(x.split("_")[1:]) for x in df.columns if x not in ["MONTH", "DATE"]  }))

In [ ]:
# Set the number of rows to use (3 years of daily data)
nr_rows = 365*3

# Select the feature columns, dropping 'DATE' and 'MONTH'
X_data = df.loc[:nr_rows].drop(columns=["DATE", "MONTH"])

In [ ]:
# Show the shape of the feature matrix
X_data.shape

In [ ]:
# Print all column names that contain 'BASEL'
print([ item for item in df.columns if "BASEL" in item])

In [ ]:
# Select the target variable: next day's sunshine hours in Basel
y_data = df.loc[1:(nr_rows+1)]["BASEL_sunshine"]

In [ ]:
# Show the data type of the target variable
y_data.dtype

In [ ]:
# Import train_test_split for splitting data into train/validation/test
from sklearn.model_selection import train_test_split

# Split the data into training and not-training sets (70% train, 30% not-train)
X_train, X_not_train, y_train, y_not_train = train_test_split(X_data, y_data, test_size=.3, random_state=20211013)

In [ ]:
# Split the not-training set into validation and test sets (each 15% of total)
X_val, X_test, y_val, y_test = train_test_split(X_not_train, y_not_train, test_size=.5, random_state=20211014)


In [ ]:
# Print the shape of the training feature matrix
print(X_train.shape)

In [ ]:
# Print the shapes of the test and validation feature matrices
print(X_test.shape, X_val.shape)

In [ ]:
# Import keras for building neural networks
from tensorflow import keras

# Define a function to create a neural network model
def create_nn():

  inputs = keras.Input(shape=(X_data.shape[1],), name="input")

  dense1 = keras.layers.Dense(100, 'relu', name="dense1")(inputs)
  dense2 = keras.layers.Dense(50, 'relu', name="dense2")(dense1)

  outputs = keras.layers.Dense(1)(dense2)

  return keras.Model(inputs=inputs, outputs=outputs, name="weather_prediction_model")

In [ ]:
# Create the neural network model
model = create_nn()

# Display the model architecture
model.summary()

In [ ]:
# Compile the model with Adam optimizer, mean squared error loss, and RMSE metric
model.compile(optimizer="adam",
              loss="mse", #MEAN SQUARED ERROR
              metrics=[keras.metrics.RootMeanSquaredError()])


In [ ]:
# Train the model on the training data for 200 epochs
history = model.fit(X_train, y_train,
                    batch_size = 32,
                    epochs=200,
                    verbose=2
                    )

In [ ]:
# Import seaborn and matplotlib for plotting
import seaborn as sns
import matplotlib.pyplot as plt

# Convert the training history to a DataFrame for analysis
history_df = pd.DataFrame.from_dict(history.history)


In [ ]:
# List all columns in the history DataFrame
history_df.columns

In [ ]:
# Plot the RMSE on the training data over epochs
sns.lineplot(data=history_df['root_mean_squared_error'])
plt.xlabel("epochs")
plt.ylabel("RMSE")

In [ ]:
# Make predictions on the training and test sets
y_train_predict = model.predict(X_train)
y_test_predict = model.predict(X_test)

In [ ]:
# Create scatter plots to compare predicted and true sunshine hours
fig, axes = plt.subplots(1, 2, figsize=(12, 6))

axes[0].scatter(y_train_predict, y_train, s=10, alpha=0.5, color="teal")
axes[0].set_title("training set")
axes[0].set_xlabel("predicted sunshine hours")
axes[0].set_ylabel("true sunshine hours")

axes[1].scatter(y_test_predict, y_test, s=10, alpha=0.5, color="teal")
axes[1].set_title("test set")
axes[1].set_xlabel("predicted sunshine hours")
axes[1].set_ylabel("true sunshine hours")


In [ ]:
# Evaluate the model performance on the training and test sets
loss_train, rmse_train = model.evaluate(X_train, y_train)
loss_test, rmse_test = model.evaluate(X_test, y_test)


In [ ]:
# Print the RMSE for the training and test sets
print("training set",rmse_train)
print("    test set",rmse_test)


# Yikes Overfitting !

## Set expectations: How difficult is the defined problem?

In [ ]:
# Use yesterday's sunshine hours as a simple baseline predictor
y_baseline_prediction = X_test["BASEL_sunshine"]

plt.figure(figsize=(5,5), dpi=100)
plt.scatter(y_baseline_prediction, y_test, s=10, alpha=.5)
plt.xlabel("sunshine hours yesterday (baseline)")
plt.ylabel("true sunshine hours")


In [ ]:
# Import mean_squared_error for performance comparison
from sklearn.metrics import mean_squared_error
import numpy as np

# Calculate the RMSE of the neural network predictions
mse_nn = mean_squared_error(y_test, y_test_predict)
rmse_nn = np.sqrt(mse_nn)

# Calculate the RMSE of the baseline predictions
mse_baseline = mean_squared_error(y_test, y_baseline_prediction)
rmse_baseline = np.sqrt(mse_baseline)


In [ ]:
# Import mean_squared_error for performance comparison
from sklearn.metrics import mean_squared_error

# Calculate the RMSE of the neural network predictions
rmse_nn = mean_squared_error(y_test, y_test_predict)


# Calculate the RMSE of the baseline predictions
rmse_baseline = mean_squared_error(y_test, y_baseline_prediction)



In [ ]:
# Print the RMSE for the neural network and baseline models
print("training set",rmse_nn)
print("baseline set",rmse_baseline)

In [ ]:
# Recreate and compile the neural network model
model = create_nn()
model.compile(optimizer='adam',
              loss='mse',
              metrics=[keras.metrics.RootMeanSquaredError()])

In [ ]:
# Train the model on the training data for 200 epochs, with validation on the validation set
history = model.fit(X_train, y_train,
                    batch_size=32,
                    epochs=200,
                    validation_data=(X_val, y_val),
                    verbose=2)

In [ ]:
# Convert the training history to a DataFrame for analysis
history_df = pd.DataFrame.from_dict(history.history)

# Plot the RMSE on the training and validation data over epochs
sns.lineplot(data=history_df[['root_mean_squared_error','val_root_mean_squared_error']])
plt.xlabel("epochs")
plt.ylabel("RMSE")

## Counteract model overfitting

reduce the number of parameters of our model

In [ ]:
# Define a new create_nn function that takes the number of nodes as arguments
def create_nn(nodes1, nodes2):

  inputs = keras.Input(shape=(X_data.shape[1],), name="input")

  dense1 = keras.layers.Dense(nodes1, 'relu', name="dense1")(inputs)
  dense2 = keras.layers.Dense(nodes2, 'relu', name="dense2")(dense1)

  outputs = keras.layers.Dense(1)(dense2)

  return keras.Model(inputs=inputs, outputs=outputs, name="weather_prediction_model")

In [ ]:
# Create a smaller neural network model with 10 and 5 nodes in the hidden layers
model = create_nn(10,5)

# Display the model architecture
model.summary()

In [ ]:
# Compile the model with Adam optimizer, mean squared error loss, and RMSE metric
model.compile(optimizer='adam',
              loss='mse',
              metrics=[keras.metrics.RootMeanSquaredError()])

# Train the model on the training data for 200 epochs, with validation on the validation set
history = model.fit(X_train, y_train,
                    batch_size = 32,
                    epochs = 200,
                    validation_data=(X_val, y_val),
                    verbose = 2)

# Convert the training history to a DataFrame for analysis
history_df = pd.DataFrame.from_dict(history.history)

# Plot the RMSE on the training and validation data over epochs
sns.lineplot(data=history_df[['root_mean_squared_error', 'val_root_mean_squared_error']])
plt.xlabel("epochs")
plt.ylabel("RMSE")



In [ ]:
# Create a larger neural network model with 100 and 50 nodes in the hidden layers
model = create_nn(100, 50)

# Compile the model with Adam optimizer, mean squared error loss, and RMSE metric
model.compile(optimizer='adam',
              loss='mse',
              metrics=[keras.metrics.RootMeanSquaredError()])


In [ ]:
# Import EarlyStopping to prevent overfitting
from tensorflow.keras.callbacks import EarlyStopping


# Set up EarlyStopping to monitor validation loss and stop training if it doesn't improve for 10 epochs
earlystop = EarlyStopping(monitor='val_loss',
                          patience=10,verbose=1)

# Train the model on the training data for 200 epochs, with validation on the validation set
# EarlyStopping will halt training early if there's no improvement
history = model.fit(X_train, y_train,
                    batch_size = 32,
                    epochs=200,
                    validation_data=(X_val, y_val),
                    callbacks=[earlystop],
                    verbose=2)

In [ ]:
# Convert the training history to a DataFrame for analysis
history_df = pd.DataFrame.from_dict(history.history)

# Plot the RMSE on the training and validation data over epochs
sns.lineplot(data=history_df[['root_mean_squared_error', 'val_root_mean_squared_error']])
plt.xlabel("epochs")
plt.ylabel("RMSE")


Further  techniques of interest:
- batchnormalisation
- dropout layers